In [35]:
#import the pandas library
try:
    import pandas as pd
except: 
    !pip install pandas --quiet
    import pandas as pd
# #import the sklearn library tools (individually)
try:
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import classification_report
except:
    !pip install scikit-learn --quiet
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import classification_report
#import the pickle library
try:
    import pickle as pkl
except:
    !pip install pickle --quiet
    import pickle as pkl
#import dataprep library eda tool
try:
    from dataprep.eda import create_report
except:
    !pip install dataprep.eda --quiet
    from dataprep.eda import create_report
#import lazypredict library classifier tool
try:
    from lazypredict.Supervised import LazyClassifier
except:
    !pip install lazypredict --quiet
    from lazypredict.Supervised import LazyClassifier
    

In [15]:
#load in training data
train_df = pd.read_csv('train.csv')

In [11]:
intial_eda_report = create_report(train_df, title = "Kaggle: Intial Spaceship Titanic EDA report") 
pkl.dump(intial_eda_report, open('intial_SpaceshipEDA.pkl','wb'))

  0%|                                                                                         | 0/1149 [00:00<…

In [13]:
intial_eda_report= pkl.load(open('intial_SpaceshipEDA.pkl','rb'))
intial_eda_report.show_browser()

In [16]:
#intial length of dataset
print(f"intial length of dataset {len(train_df)}",'\n')

#remove columns that are highly unqiue (name and ID)
train_df=train_df.drop(columns=["Name","PassengerId"])

#handle duplicate rows
#check intial number of duplicates
print(f"intial num dups: {train_df.duplicated().sum()}")
#remove duplicates
train_df = train_df.drop_duplicates()
#show the new number of duplicates
print(f"new num dups: {train_df.duplicated().sum()}",'\n')

#handle NaN values
#check intial number of NaN
print(f"intial num NaN:{train_df.isna().sum()}")
#remove NaN values
train_df.dropna(inplace=True)
#show new number of NaN values
print(f"new num NaN:{train_df.isna().sum()}",'\n')

#fix index on dataframe
train_df.reset_index(drop=True)

#new length of dataset
print(f"new length of dataset {len(train_df)}",'\n')

intial length of dataset 8693 

intial num dups: 15
new num dups: 0 

intial num NaN:HomePlanet      201
CryoSleep       217
Cabin           193
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Transported       0
dtype: int64
new num NaN:HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
dtype: int64 

new length of dataset 6755 



In [6]:
#current dataset (first 5 rows)
train_df.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,B/0/P,TRAPPIST-1e,39.00,False,0.00,0.00,0.00,0.00,0.00,False
1,Earth,False,F/0/S,TRAPPIST-1e,24.00,False,109.00,9.00,25.00,549.00,44.00,True
2,Europa,False,A/0/S,TRAPPIST-1e,58.00,True,43.00,3576.00,0.00,6715.00,49.00,False
3,Europa,False,A/0/S,TRAPPIST-1e,33.00,False,0.00,1283.00,371.00,3329.00,193.00,False
4,Earth,False,F/1/S,TRAPPIST-1e,16.00,False,303.00,70.00,151.00,565.00,2.00,True


Transform data

In [17]:
#turns a binary value into a 1 or 0
def to_binary(df,column):
    #set false to be 0 and true to be 1
    mapping = {False: 0, True: 1}
    #change dataframe to reflect the numerical (binary) representation
    df[column] = df[column].map(mapping)
    return df
#binary columns in dataset
binary_columns = [
    "CryoSleep", #turn cryp sleep 1 and 0
    "VIP", #turn vip to 1 and 0
    "Transported" #turn transported into 1 and 0
]
#goes through all binary columns in dataset and changes them to 1 and 0 in are data
for column in binary_columns:
    train_df = to_binary(train_df,column)


In [32]:
#currently not in use

# #combine money spent across all services into 1 column
# #all the luxury services that cost money in the data
# luxury_services = [
#     "RoomService",
#     "FoodCourt",
#     "ShoppingMall",
#     "Spa",
#     "VRDeck"
# ]
# #added all money spent and create new column "money_spent" 
# train_df["money_spent"] = train_df[luxury_services].sum(axis=1)
# #remove all of the luxury services columns
# train_df = train_df.drop(columns = luxury_services)

In [18]:
#split cabin into it three respective parts of deck, number and side
train_df[["deck","cabin_number","side"]] = train_df["Cabin"].str.split('/',expand=True)
train_df["side"]=train_df["side"].map({"P":0,"S":1})
train_df = train_df.drop(columns = "Cabin")

In [19]:
#show data after transformation
train_df.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,deck,cabin_number,side
0,Europa,0,TRAPPIST-1e,39.00,0,0.00,0.00,0.00,0.00,0.00,0,B,0,0
1,Earth,0,TRAPPIST-1e,24.00,0,109.00,9.00,25.00,549.00,44.00,1,F,0,1
2,Europa,0,TRAPPIST-1e,58.00,1,43.00,3576.00,0.00,6715.00,49.00,0,A,0,1
3,Europa,0,TRAPPIST-1e,33.00,0,0.00,1283.00,371.00,3329.00,193.00,0,A,0,1
4,Earth,0,TRAPPIST-1e,16.00,0,303.00,70.00,151.00,565.00,2.00,1,F,1,1


In [20]:
Post_Transformation_eda_report = create_report(train_df, title = "Kaggle: Post-Transformation Spaceship Titanic EDA report") 
pkl.dump(Post_Transformation_eda_report, open('Post_Transformation_SpaceshipEDA.pkl','wb'))

  0%|                                                                                         | 0/1897 [00:00<…

In [21]:
pkl.load(open('Post_Transformation_SpaceshipEDA.pkl','rb'))
Post_Transformation_eda_report.show_browser()

In [23]:
#seperate features and label
feature_names = [
    "HomePlanet",
    "CryoSleep",
    "Destination",
    "Age",
    "VIP",
    "RoomService",
    "FoodCourt",
    "ShoppingMall",
    "Spa",
    "VRDeck",
    "deck",
    "cabin_number",
    "side"
]
label_name = "Transported"
#seperate training and validation data
train_features, val_features, train_label, val_label=train_test_split(train_df[feature_names],train_df[label_name], test_size=0.2, random_state=42)
#save all datasets
pkl.dump(train_features,open("train_features.pkl","wb"))
pkl.dump(train_label,open("train_label.pkl","wb"))
pkl.dump(val_features,open("val_features.pkl","wb"))
pkl.dump(val_label,open("val_label.pkl","wb"))

In [24]:
#load saved datasets
train_features  =pkl.load(open("train_features.pkl","rb"))
train_label = pkl.load(open("train_label.pkl","rb"))
val_features = pkl.load(open("val_features.pkl","rb"))
val_label = pkl.load(open("val_label.pkl","rb"))

In [25]:
train_features.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,deck,cabin_number,side
3099,Earth,1,PSO J318.5-22,28.00,0,0.00,0.00,0.00,0.00,0.00,G,546,0
5868,Europa,0,55 Cancri e,37.00,0,0.00,90.00,0.00,3282.00,11.00,B,239,1
7888,Earth,1,TRAPPIST-1e,20.00,0,0.00,0.00,0.00,0.00,0.00,G,1349,1
3929,Earth,1,TRAPPIST-1e,13.00,0,0.00,0.00,0.00,0.00,0.00,G,688,1
5239,Mars,1,55 Cancri e,53.00,0,0.00,0.00,0.00,0.00,0.00,F,1161,0


Normalization

In [26]:
#onehot encode the nominal features in the dataset

#nominal features in dataset to be one hot encoded
nominal_columns = [
    "HomePlanet", #onehot encode homeplanet
    "Destination", #onehot encode destination
    "deck"
]

#fit onehot encoder to nominal data
def fit_one_hot_encoding(df,nominal_features):
    encoder = OneHotEncoder(handle_unknown='ignore')
    #fit encoder on training data
    encoder.fit(df[nominal_features])
    #save fitted onehot encoder
    pkl.dump(encoder,open("OneHotEncoder.pkl","wb"))

#transform nominal data to one hot encoding with encoder
def to_onehot(df,nominal_features,encoder_file):
    #load existing encoder 
    encoder = pkl.load(open(encoder_file,"rb"))
    #transform nominal features to onehot
    onehot = encoder.transform(df[nominal_features])
    #create matrix of nominal feature
    onehot_df = pd.DataFrame(onehot.todense(), columns = encoder.get_feature_names_out())
    #update dataframe
    df = df.drop(columns=nominal_features)
    df.reset_index(drop=True, inplace=True)
    df = pd.concat([df,onehot_df],axis=1)
    #save changes and return dataset
    return df

fit_one_hot_encoding(train_features, nominal_columns)
train_features = to_onehot(train_features,nominal_columns,"OneHotEncoder.pkl")

In [27]:
#minmax scale numerical features in the datasett

#numerical features in the dataset to be encoded
numerical_columns = [
    "Age", #minmax age
    "RoomService",
    "FoodCourt",
    "ShoppingMall",
    "Spa",
    "VRDeck",
    "cabin_number"
]

#fit scaler to numerical data
def fit_MinMax(df,numerical_features):
    Scaler = MinMaxScaler()
    #fit scaler on training data
    Scaler.fit(df[numerical_features])
    #save fitted scaler
    pkl.dump(Scaler,open("MinMaxScaler.pkl","wb"))

def to_MinMax(df,numerical_features,scaler_file):
    #load existing scaler 
    scaler = pkl.load(open(scaler_file,"rb"))
    #transform numerical features to MinMax
    df[numerical_features] = scaler.transform(df[numerical_features])
    #save changes and return dataset
    return df

fit_MinMax(train_features,numerical_columns)
train_features = to_MinMax(train_features,numerical_columns,"MinMaxScaler.pkl")

In [28]:
norm_train_features = train_features
norm_train_features.head()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,cabin_number,side,...,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,deck_A,deck_B,deck_C,deck_D,deck_E,deck_F,deck_G,deck_T
0,1,0.35,0,0.00,0.00,0.00,0.00,0.00,0.29,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00
1,0,0.47,0,0.00,0.00,0.00,0.15,0.00,0.13,1,...,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00
2,1,0.25,0,0.00,0.00,0.00,0.00,0.00,0.71,1,...,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00
3,1,0.16,0,0.00,0.00,0.00,0.00,0.00,0.36,1,...,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00
4,1,0.67,0,0.00,0.00,0.00,0.00,0.00,0.61,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00


In [29]:
val_features = to_onehot(val_features,nominal_columns,"OneHotEncoder.pkl")
val_features = to_MinMax(val_features,numerical_columns,"MinMaxScaler.pkl")
norm_val_features = val_features

In [30]:
#lazy predict
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(norm_train_features, norm_val_features, train_label, val_label)

 97%|███████████████████████████████████████████████████████████████████████████████▏  | 28/29 [00:19<00:00,  1.42it/s]

[LightGBM] [Info] Number of positive: 2705, number of negative: 2699
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.042088 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1656
[LightGBM] [Info] Number of data points in the train set: 5404, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500555 -> initscore=0.002221
[LightGBM] [Info] Start training from score 0.002221


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:20<00:00,  1.43it/s]


In [31]:
print(models)

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
SVC                                0.81               0.81     0.81      0.81   
RandomForestClassifier             0.81               0.81     0.81      0.81   
ExtraTreesClassifier               0.81               0.81     0.81      0.81   
SGDClassifier                      0.81               0.81     0.81      0.81   
LGBMClassifier                     0.81               0.81     0.81      0.81   
NuSVC                              0.81               0.81     0.81      0.81   
LogisticRegression                 0.80               0.80     0.80      0.80   
XGBClassifier                      0.80               0.80     0.80      0.80   
AdaBoostClassifier                 0.80               0.80     0.80      0.80   
KNeighborsClassifier               0.80               0.80     0.80      0.80   
CalibratedClassifierCV      